In [1]:
import numpy as np
import genesis as gs
import time

# GPU 백엔드 사용
gs.init(backend=gs.gpu)

# Scene 생성
scene = gs.Scene(
    viewer_options=gs.options.ViewerOptions(
        camera_pos=(0, -3.5, 2.5),
        camera_lookat=(0.0, 0.0, 0.5),
        camera_fov=30,
        max_FPS=60,
    ),
    sim_options=gs.options.SimOptions(dt=0.01),
    show_viewer=True,
)

# 객체 추가
plane = scene.add_entity(gs.morphs.Plane())
table = scene.add_entity(
    gs.morphs.URDF(
        file='/home/wockd/Documents/table+aloha+cube/table_model.urdf',
        fixed=True,
        pos=(0.95, 0, 0),
    ),
)
cube = scene.add_entity(
    gs.morphs.URDF(
        file='/home/wockd/Documents/table+aloha+cube/cube.urdf',
        fixed=False,
        pos=(0.81, -0.05, 1.035),  # 테이블 위 물체 위치
    ),
)
arx5 = scene.add_entity(
    gs.morphs.URDF(
        file='/home/wockd/Documents/table+aloha+cube/arx5_description_isaac.urdf',
        fixed=True,
    ),
)

scene.build()

# 오른쪽 팔 관절만 사용
fr_joint_names = [f"fr_joint{i}" for i in range(1, 9)]
dofs_idx = [arx5.get_joint(name).dof_idx_local for name in fr_joint_names]

# 제어 게인 설정
revolute_joints = dofs_idx[:6]  # 회전 관절 (1-6)
gripper_joints = dofs_idx[6:]   # 그리퍼 관절 (7-8)

# Position gains
arx5.set_dofs_kp(
    kp=np.array([4000] * len(revolute_joints) + [2000] * len(gripper_joints)),
    dofs_idx_local=dofs_idx,
)

# Velocity gains
arx5.set_dofs_kv(
    kv=np.array([400] * len(revolute_joints) + [200] * len(gripper_joints)),
    dofs_idx_local=dofs_idx,
)

# Force limits
force_limit_revolute = 80
force_limit_gripper = 40
arx5.set_dofs_force_range(
    lower=np.array([-force_limit_revolute] * len(revolute_joints) + 
                   [-force_limit_gripper] * len(gripper_joints)),
    upper=np.array([force_limit_revolute] * len(revolute_joints) + 
                   [force_limit_gripper] * len(gripper_joints)),
    dofs_idx_local=dofs_idx,
)

# 물체 잡기 시퀀스
print("Starting pick sequence...")
for i in range(1000):
    if i < 200:  # 초기 자세
        position = np.zeros(8)
        position[0] = 0.5  # 팔 회전
        position[1] = 3.2  # 첫번째 관절 앞으로
        position[2] = 2.7  # 팔꿈치? 부분 펴기
        position[3] = -0.2  # 팔꿈치 구부리기
        position[6:8] = 0.04  # 그리퍼 열기
        arx5.control_dofs_position(position, dofs_idx)
    
    elif i < 400:  # 물체 위로 이동
        position = np.zeros(8)
        position[0] = 0.5  # 팔 회전
        position[1] = 3.2  # 첫번째 관절 앞으로
        position[2] = 2.7  # 팔꿈치? 부분 펴기
        position[3] = -0.7  # 팔꿈치 더 구부리기
        position[4] = 0.4  # 손목 조정
        position[6:8] = 0.04  # 그리퍼 유지
        arx5.control_dofs_position(position, dofs_idx)
    
    elif i < 600:  # 그리퍼 닫기
        position = np.zeros(8)
        position[0] = 0.5  # 팔 회전
        position[1] = 3.2
        position[2] = 2.7
        position[3] = -0.7
        position[4] = 0.4
        position[5] = 0.3
        position[6:8] = 0.015  # 그리퍼 닫기
        arx5.control_dofs_position(position, dofs_idx)
    
    elif i < 800:  # 물체 들어올리기
        position = np.zeros(8)
        position[0] = -0.5  # 팔 회전
        position[1] = 1.5
        position[2] = 1.5
        position[3] = -0.8
        position[4] = 0.4
        position[5] = 0.3
        position[6:8] = 0.015  # 그리퍼 유지
        arx5.control_dofs_position(position, dofs_idx)
    
    # 100 스텝마다 상태 출력
    if i % 100 == 0:
        print(f"Step {i}")
        print("Joint positions:", arx5.get_dofs_position(dofs_idx))
    
    scene.step()
    time.sleep(0.01)  # 동작 속도 조절

print("Pick sequence completed")


[I 01/18/25 17:13:57.404 6265] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [17:13:59] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [17:13:59] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [17:13:59] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [17:13:59] [INFO] Running on [NVIDIA GeForce RTX 3080] with backend gs.cuda. Device memory: 9.77 GB.
[Genesis] [17:14:00] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [17:14:00] [INFO] Scene <21bd6d8> created.
[Genesis] [17:14:00] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <e891702>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [17:14:00] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <573dd2c>, morph: <gs.morphs.URDF(file='/home/wockd/Documents/table+aloha+cube/table_model.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [17:

Starting pick sequence...
Step 0
Joint positions: tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


[Genesis] [17:14:16] [INFO] Running at 84.42 FPS.
[Genesis] [17:14:16] [INFO] Running at 81.46 FPS.
[Genesis] [17:14:16] [INFO] Running at 78.50 FPS.
[Genesis] [17:14:16] [INFO] Running at 76.42 FPS.
[Genesis] [17:14:16] [INFO] Running at 74.33 FPS.
[Genesis] [17:14:16] [INFO] Running at 72.58 FPS.
[Genesis] [17:14:16] [INFO] Running at 70.84 FPS.
[Genesis] [17:14:16] [INFO] Running at 69.30 FPS.
[Genesis] [17:14:16] [INFO] Running at 67.90 FPS.
[Genesis] [17:14:16] [INFO] Running at 66.75 FPS.
[Genesis] [17:14:16] [INFO] Running at 65.45 FPS.
[Genesis] [17:14:16] [INFO] Running at 64.41 FPS.
[Genesis] [17:14:16] [INFO] Running at 63.50 FPS.
[Genesis] [17:14:17] [INFO] Running at 62.63 FPS.
[Genesis] [17:14:17] [INFO] Running at 61.85 FPS.
[Genesis] [17:14:17] [INFO] Running at 61.07 FPS.
[Genesis] [17:14:17] [INFO] Running at 60.34 FPS.
[Genesis] [17:14:17] [INFO] Running at 59.70 FPS.
[Genesis] [17:14:17] [INFO] Running at 59.10 FPS.
[Genesis] [17:14:17] [INFO] Running at 58.54 FPS.


Step 100
Joint positions: tensor([ 4.9993e-01,  3.7073e+00,  3.7660e+00, -1.9967e-01,  1.9826e-05,
        -4.1999e-05,  3.9998e-02,  4.0193e-02], device='cuda:0')


[Genesis] [17:14:19] [INFO] Running at 42.60 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.37 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.37 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.50 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.56 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.69 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.89 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.94 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.97 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.97 FPS.
[Genesis] [17:14:19] [INFO] Running at 42.96 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.11 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.18 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.15 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.22 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.28 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.24 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.35 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.41 FPS.
[Genesis] [17:14:19] [INFO] Running at 43.43 FPS.


...

Step 800
Joint positions: tensor([-0.5000,  1.5011,  1.4988, -0.8002,  0.4000,  0.3000,  0.0150,  0.0150],
       device='cuda:0')


[Genesis] [17:14:35] [INFO] Running at 45.89 FPS.
[Genesis] [17:14:35] [INFO] Running at 45.89 FPS.
[Genesis] [17:14:35] [INFO] Running at 45.82 FPS.
[Genesis] [17:14:35] [INFO] Running at 45.70 FPS.
[Genesis] [17:14:35] [INFO] Running at 45.85 FPS.
[Genesis] [17:14:35] [INFO] Running at 45.96 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.13 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.24 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.34 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.50 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.64 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.72 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.80 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.65 FPS.
[Genesis] [17:14:35] [INFO] Running at 46.66 FPS.
[Genesis] [17:14:36] [INFO] Running at 46.56 FPS.
[Genesis] [17:14:36] [INFO] Running at 46.64 FPS.
[Genesis] [17:14:36] [INFO] Running at 46.74 FPS.
[Genesis] [17:14:36] [INFO] Running at 46.84 FPS.
[Genesis] [17:14:36] [INFO] Running at 46.94 FPS.


Step 900
Joint positions: tensor([-0.5000,  1.5011,  1.4988, -0.8002,  0.4000,  0.3000,  0.0150,  0.0150],
       device='cuda:0')


[Genesis] [17:14:38] [INFO] Running at 43.16 FPS.
[Genesis] [17:14:38] [INFO] Running at 43.27 FPS.
[Genesis] [17:14:38] [INFO] Running at 43.24 FPS.
[Genesis] [17:14:38] [INFO] Running at 42.95 FPS.
[Genesis] [17:14:38] [INFO] Running at 42.45 FPS.
[Genesis] [17:14:38] [INFO] Running at 42.25 FPS.
[Genesis] [17:14:38] [INFO] Running at 42.54 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.87 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.63 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.56 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.53 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.26 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.09 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.07 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.02 FPS.
[Genesis] [17:14:38] [INFO] Running at 41.02 FPS.
[Genesis] [17:14:38] [INFO] Running at 40.79 FPS.
[Genesis] [17:14:38] [INFO] Running at 40.79 FPS.
[Genesis] [17:14:38] [INFO] Running at 40.76 FPS.
[Genesis] [17:14:38] [INFO] Running at 40.75 FPS.


Pick sequence completed
